<h1 align=center>Robots! Robots! RA! RA! RA!</h1>
<h4 align=center>Katherine Scott / @kscottz / katherine@tempoautomation.com</h4>
<img src="/files/images/bender.jpg" height=400 align=center>


<h1 align=center>Why should you listen to me?</h1>
<img src="files/images/youngkat.jpg" width=400 align=left>
<img src="files/images/hmd.jpg" width=400 align=center>
<img src="files/images/essess.jpg" width=400 align=right>



<h1 align=center>I am also doing this full time at Tempo Automation</h1>
<img src="/files/images/tempo.jpg">

<blink><h2 style="color:#F00"><i>Shameless plug -- I am hiring. Also we can probably make your circuit boards faster and cheaper.</h2></i></blink>

<h1 align=center>This is what that robot does.</h1>
<img src="/files/images/smt.jpg" align=left>
<img src="/files/images/pcb.jpg" align=right>




<h1 align=center> Before we talk about robots let's talk about the web </h1>
<img src="/files/images/wheel.jpg">
<h2>
<ul>
<li>Who writes a web server from first principals?</li>
<br>
<li>That one person did it once, 'cause it is hard.</li>
<br>
<li>We take it at a given when we go to do work.</li>
<br>
<li><b>SO WHY WOULD YOU EXPECT TO WRITE A ROBOT FROM FIRST PRINCIPALS?</b></li>
<br>
<li> Wheels -- do not require re-invention </li>
</h2>
</ul>

<h1 align=center> Good News -- There's robotics package for Python</h1>
<img src="/files/images/ros.png" align=center>
<h2>
<ul align=center>
<li>Runs some of the most advanced robots.</li>
<br>
<li>Plays well on Linux and runs on x86 and ARM</li>
<br>
<li><B>REALLY REALLY AWESOME FOSS COMMUNITY</B></li>
<br>
<li>Reasonably mature and battle hardened.</li>
<br>
<li>Let's C/C++/Lisp/Python/Java/JS play well together </li>
</h2>
</ul>

<h1 align=center> ROS -- So what is it? </h1>

<img src="/files/images/mind_blown.gif" align=center>
<h2>
<ul align=center>
<li>Package manager, build system (catkin), and deployment all in one.</li>
<br>
<li>Unified "system bus" for message passing</li>
<br>
<li>Every utility you could ever think of.</li>
<br>
<li>Plugin framework for adding robot modules.</li>
<br>
<li>Standard libraries for handling images, transforms, 3D models, etc.</li>
</h2>
</ul>


<h1 align=center> I could go on chatting ...but let's just do it instead.</h1>
<br>
<img src="/files/images/twitchy.jpg" width=400 align=left>
<img src="/files/images/kuka.jpg" width=400 align=right>
<br>
<br>


